n-gram是一种统计语言模型，用于预测文本中下一个项目的概率。在自然语言处理中，n-gram可以用来构建文本的特征。n-gram模型基于假设，一个词的出现概率与它前面的n-1个词有关。n-gram模型可以捕捉到文本中的局部模式，对于文本分类、拼写检查、语音识别等任务非常有用。

**原理：**
假设有一个文本序列："I have a dream"，如果我们使用bigram（2-gram）模型，就可以生成以下序列：
- (I, have)
- (have, a)
- (a, dream)

如果是trigram（3-gram）模型，则会生成：
- (I, have, a)
- (have, a, dream)

n-gram模型会统计这些词组在语料库中出现的频率，然后用来预测下一个词或者进行其他NLP任务。

**构建n-gram特征的步骤：**
1. 选择n-gram的大小（n值）。
2. 清洗和预处理文本（如分词、去除停用词等）。
3. 根据n-gram大小，将文本分割成一系列的n-gram。
4. 计算每个n-gram的出现频率。
5. 使用n-gram特征进行机器学习建模。


In [1]:
from collections import defaultdict
import re

def tokenize(text):
    # 简单的分词，可以根据需要使用更复杂的分词方法,用于将文本分割成单词
    return re.findall(r'\w+', text.lower())

def create_n_grams(tokens, n):
    # 用于生成n-gram
    n_grams = zip(*[tokens[i:] for i in range(n)])
    return [' '.join(n_gram) for n_gram in n_grams]

def n_gram_features(text, n=2):
    # 用于计算每个n-gram的出现次数，并返回一个特征字典。
    tokens = tokenize(text)
    print('tokens:',tokens)
    n_grams = create_n_grams(tokens, n)
    features = defaultdict(int)
    
    for n_gram in n_grams:
        features[n_gram] += 1
        
    return features

# 示例文本
text = "I have a dream that one day this nation will rise up."

# 生成bigram特征
bigram_features = n_gram_features(text, n=2)
print(bigram_features)

# 生成trigram特征
trigram_features = n_gram_features(text, n=3)
print(trigram_features)

tokens: ['i', 'have', 'a', 'dream', 'that', 'one', 'day', 'this', 'nation', 'will', 'rise', 'up']
defaultdict(<class 'int'>, {'i have': 1, 'have a': 1, 'a dream': 1, 'dream that': 1, 'that one': 1, 'one day': 1, 'day this': 1, 'this nation': 1, 'nation will': 1, 'will rise': 1, 'rise up': 1})
tokens: ['i', 'have', 'a', 'dream', 'that', 'one', 'day', 'this', 'nation', 'will', 'rise', 'up']
defaultdict(<class 'int'>, {'i have a': 1, 'have a dream': 1, 'a dream that': 1, 'dream that one': 1, 'that one day': 1, 'one day this': 1, 'day this nation': 1, 'this nation will': 1, 'nation will rise': 1, 'will rise up': 1})


构建特征矩阵是将文本数据转换为机器学习算法可以处理的数值型格式的过程。在自然语言处理中，一个常见的方法是使用词袋模型（Bag of Words, BoW），它将文本转换为单词出现次数的向量表示。对于bigram特征，这个过程是类似的，只是我们不仅考虑单个单词，还考虑单词对（即bigram）的出现次数。

以下是构建bigram特征矩阵的步骤：

1. **创建词汇表**：首先，需要从所有文档中提取所有出现过的bigram，并创建一个词汇表。词汇表是一个字典，它将每个bigram映射到一个唯一的整数索引。

2. **计算bigram频率**：对于每个文档，计算词汇表中每个bigram的出现次数。这可以通过遍历文档中的每个单词对，并使用词汇表中的索引来实现。

3. **构建特征向量**：对于每个文档，使用词汇表中的索引创建一个特征向量。向量的长度等于词汇表的大小，每个元素的值是对应bigram在文档中的出现次数。

4. **构建特征矩阵**：对于所有文档，重复上述过程，最终得到一个特征矩阵，其中每一行代表一个文档，每一列代表一个bigram特征。

5. **标准化/归一化**（可选）：为了消除文档长度对模型的影响，可以选择对特征向量进行标准化（例如，转换为TF-IDF值）或归一化（例如，每个特征向量的长度为1）。





在这个例子中，`CountVectorizer`自动为我们完成了创建词汇表、计算bigram频率和构建特征矩阵的步骤。`X`是一个稀疏矩阵，其中每一行对应一个文档，每一列对应一个bigram特征，矩阵中的值是bigram在文档中的出现次数。

请注意，`CountVectorizer`返回的是一个稀疏矩阵，这是处理大型文本数据集时的一种内存高效的方式。在实际应用中，你可以根据需要将稀疏矩阵转换为密集矩阵，或者直接在稀疏矩阵上进行机器学习建模。


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# 示例文档
documents = [
    "I have a dream  have a dream",
    "I have a pen",
    "I think therefore I am",
    "I am machine learning",
    # ... 更多文档
]

# 初始化CountVectorizer，设置ngram_range为2-gram
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 2))
print('vectorizer:',vectorizer)

# 转换文本数据为特征矩阵
X = vectorizer.fit_transform(documents)

# 获取特征词汇（bigram词汇表）
feature_names = vectorizer.get_feature_names_out()

# 打印特征矩阵（稀疏格式）
print("Feature matrix (terms x documents):")
print(X.toarray())

# 打印bigram词汇表
print("Bigram vocabulary (terms):")
print(feature_names)

vectorizer: CountVectorizer(ngram_range=(2, 2))
Feature matrix (terms x documents):
[[0 1 2 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 0 1 1]
 [1 0 0 0 1 0 0]]
Bigram vocabulary (terms):
['am machine' 'dream have' 'have dream' 'have pen' 'machine learning'
 'therefore am' 'think therefore']


以下是一个简单的Python代码示例，展示了如何使用bigram特征来训练一个逻辑回归分类器：

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 示例文档
documents = [
    "I have a dream",
    "I have a pen",
    "I think therefore I am",
    "I am machine learning",
    # ... 更多文档
]

# 分类标签
labels = [0, 1, 0, 1]  # 假设0和1是两类标签

# 构建一个管道，包括词袋模型和逻辑回归分类器
model = make_pipeline(CountVectorizer(analyzer='word', ngram_range=(2, 2)), LogisticRegression())

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(documents, labels, test_size=0.2, random_state=42)

# 训练模型
model.fit(X_train, y_train)

# 预测测试集
predictions = model.predict(X_test)

# 评估模型
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.0
